<a href="https://colab.research.google.com/github/palakagl/NLP/blob/main/Wysdom_BaseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import io
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from google.colab import files
uploaded = files.upload()

Saving public_data.csv to public_data.csv


In [3]:
df = pd.read_csv(io.BytesIO(uploaded['public_data.csv']))

In [4]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8884 entries, 0 to 8883
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       8884 non-null   int64 
 1   message  8884 non-null   object
 2   label    8884 non-null   object
dtypes: int64(1), object(2)
memory usage: 208.3+ KB
None
      id                                            message             label
0  10480                           start the robotic vacuum      iot_cleaning
1   4210          i am sorry but i think you are not right.    general_negate
2   2443  what is the time difference between california...  datetime_convert
3   5869                              tell me my list names       lists_query
4   2801                 beep when i get an email from john       email_query


In [5]:
X = df['message']
y = df['label']

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(7107,)
(7107,)
(1777,)
(1777,)


In [12]:
vectorizer = CountVectorizer(min_df=.01, max_df=.8, ngram_range=[1,3], max_features=550, stop_words='english')

pipe = Pipeline([('vec', vectorizer),  ('clf', RandomForestClassifier(random_state=223))])

pipe.fit(X_train, y_train)

Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.8, max_features=550, min_df=0.01,
                                 ngram_range=[1, 3], stop_words='english')),
                ('clf', RandomForestClassifier(random_state=223))])

In [13]:
pred_val = pipe.predict(X_val)
print(confusion_matrix(y_val, pred_val))
print(classification_report(y_val, pred_val))

[[21  1  7 ...  0  0  0]
 [ 5 12  6 ...  0  0  0]
 [ 4  1 14 ...  0  0  1]
 ...
 [ 0  0  0 ... 24  0  0]
 [ 0  0  0 ...  0 27  0]
 [ 0  0  0 ...  0  0 12]]
                          precision    recall  f1-score   support

             alarm_query       0.70      0.72      0.71        29
            alarm_remove       0.86      0.52      0.65        23
               alarm_set       0.52      0.48      0.50        29
       audio_volume_down       0.56      0.42      0.48        12
       audio_volume_mute       0.00      0.00      0.00        23
         audio_volume_up       0.69      0.38      0.49        24
          calendar_query       0.33      0.06      0.11        31
         calendar_remove       0.59      0.77      0.67        31
            calendar_set       0.43      0.29      0.35        34
          cooking_recipe       0.10      0.09      0.09        35
        datetime_convert       0.80      0.62      0.70        13
          datetime_query       0.46      0.40      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
ari = adjusted_rand_score(y_val, pred_val)
ami = adjusted_mutual_info_score(y_val, pred_val, average_method='arithmetic')

print("ARI: {}".format(ari))
print("AMI: {}".format(ami))

ARI: 0.08401591777160755
AMI: 0.4536053837690067


## Kaggle Predictions

In [15]:
from google.colab import files
uploaded = files.upload()

Saving input_data.csv to input_data.csv


In [16]:
df_test = pd.read_csv(io.BytesIO(uploaded['input_data.csv']))

In [17]:
#df_test = pd.read_csv('input_data.csv')
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2221 entries, 0 to 2220
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2221 non-null   int64 
 1   message  2221 non-null   object
dtypes: int64(1), object(1)
memory usage: 34.8+ KB


,id,message
0,10873,list the current price of microsoft shares
1,8491,where is the nearest smoke shop
2,4549,please start over.
3,5948,read my lists to me
4,1663,mute volume now


In [18]:
pred_test = pipe.predict(df_test['message'])
pred_test

array(['lists_query', 'general_affirm', 'iot_cleaning', ...,
       'iot_hue_lightdim', 'email_sendemail', 'general_affirm'],
      dtype=object)

In [19]:

my_submission = pd.DataFrame({'Id': df_test['id'], 'label': pred_test})
print(my_submission.head())

      Id              label
0  10873        lists_query
1   8491     general_affirm
2   4549       iot_cleaning
3   5948     general_affirm
4   1663  audio_volume_down


In [20]:
# NOTE: after saving the CSV file, be sure to zip the file before submitting to the competition website!
my_submission.to_csv('Answers_RF.csv', index=False)